In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

# Assingment 1 - Web Scraping Bumeran - Luis Ricardo Céspedes Espinoza

Los pasos del web scrapping están en el segmento 1, realizado por Arianna Zavala

In [193]:
url_base = "https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html"

In [201]:
driver = webdriver.Chrome()
driver.get("https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html")

# STAGE 1: We get all the links of the job offers in all the pages


In [ ]:
# --- 1. Detectar el número máximo de páginas ---
driver.get(url_base)
time.sleep(8)
pages = driver.find_elements(By.XPATH, "//a[contains(@href, 'page=')]")
page_numbers = []

for p in pages:
    href = p.get_attribute("href")
    if href and "page=" in href:
        try:
            num = int(href.split("page=")[-1])
            page_numbers.append(num)
        except:
            pass

max_page = max(page_numbers) if page_numbers else 1
print(f"📊 Se detectaron {max_page} páginas de avisos")

# --- 2. Recorrer cada página y extraer solo los avisos ---
avisos = []

for page in range(1, max_page + 1):
    if page == 1:
        url = url_base
    else:
        url = f"{url_base}?page={page}"

    print(f"\n📄 Procesando página {page}: {url}")
    driver.get(url)
    time.sleep(6)

    cards = driver.find_elements(By.CSS_SELECTOR, "#listado-avisos a")

    for card in cards:
        link = card.get_attribute("href")
        # filtro: solo avisos reales
        if link and "/empleos/" in link and link.endswith(".html"):
            if link not in avisos:
                avisos.append(link)
                print(f"Aviso {len(avisos)}: {link}")

driver.quit()

# --- 3. Guardar en CSV ---
with open("avisos_bumeran.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["N°", "Link"])
    for i, link in enumerate(avisos, 1):
        writer.writerow([i, link])

print(f"\n💾 Se guardaron {len(avisos)} avisos en 'avisos_bumeran.csv'")

📊 Se detectaron 6 páginas de avisos

📄 Procesando página 1: https://www.bumeran.com.pe/en-lima/empleos-area-tecnologia-sistemas-y-telecomunicaciones-subarea-programacion-full-time-publicacion-menor-a-15-dias.html
Aviso 1: https://www.bumeran.com.pe/empleos/software-engineer-senior-grupo-gloria-1117976663.html
Aviso 2: https://www.bumeran.com.pe/empleos/programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html
Aviso 3: https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
Aviso 4: https://www.bumeran.com.pe/empleos/analista-programador-sede-ate-bumeran-selecta-1117969140.html
Aviso 5: https://www.bumeran.com.pe/empleos/trainee-programador-jr-building-software-1117968407.html
Aviso 6: https://www.bumeran.com.pe/empleos/desarrollador-full-stack-node.js-c-java-angularjs-ibr-peru-s.a.-1117967655.html
Aviso 7: https://www.bumeran.com.pe/empleos/java-developer-senior-inetum-peru-1117979249.html
Aviso 8: https://www.bumeran.com.pe/empleos/senior-react-developer-enc

# STAGE 2: We extract the job details from every URL

In [199]:
# --- 3. Entrar a cada aviso y extraer Job Title ---
datos = []

for i, link in enumerate(avisos, 1):
    print(f"\n🔎 Procesando aviso {i}: {link}")
    driver.get(link)
    time.sleep(9)

    try:
        job_title = driver.find_element(By.TAG_NAME, "h1").text
    except NoSuchElementException:
        job_title = ""

    datos.append([i, link, job_title])
    print(f"➡️ {job_title}")

driver.quit()

# --- 4. Guardar en CSV final ---
with open("avisos_bumeran_detalle.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["N°", "Link", "Job Title"])
    writer.writerows(datos)

print("\n💾 Se exportaron todos los Job Titles en 'avisos_bumeran_detalle.csv'")


🔎 Procesando aviso 1: https://www.bumeran.com.pe/empleos/software-engineer-senior-grupo-gloria-1117976663.html
➡️ Software Engineer Senior

🔎 Procesando aviso 2: https://www.bumeran.com.pe/empleos/programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html
➡️ Programador Frontend Senior

🔎 Procesando aviso 3: https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
➡️ Analista Programador Java

🔎 Procesando aviso 4: https://www.bumeran.com.pe/empleos/analista-programador-sede-ate-bumeran-selecta-1117969140.html
➡️ Analista Programador - Sede ATE

🔎 Procesando aviso 5: https://www.bumeran.com.pe/empleos/trainee-programador-jr-building-software-1117968407.html
➡️ Trainee Programador Jr

🔎 Procesando aviso 6: https://www.bumeran.com.pe/empleos/desarrollador-full-stack-node.js-c-java-angularjs-ibr-peru-s.a.-1117967655.html
➡️ Desarrollador Full Stack Node.js, C#, Java, AngularJS

🔎 Procesando aviso 7: https://www.bumeran.com.pe/empleos/java-developer-senior-inetum-

We can also find elements by their ID. We can find an element's ID by once again inspecting the page. We must keep in mind that an ID may not always be available, as it must be manually set by the programmer of the webpage. In this case, the dropdown menus for filtering by geography do have IDs available.

In [203]:
datos = []

for i, link in enumerate(avisos, 1):
    print(f"\n🔎 Procesando aviso {i}: {link}")
    driver.get(link)
    time.sleep(9)

    # Job Title
    try:
        job_title = driver.find_element(By.TAG_NAME, "h1").text
    except NoSuchElementException:
        job_title = ""

    # Distrito
    distrito = ""
    ubicaciones = driver.find_elements(By.TAG_NAME, "h2")
    for u in ubicaciones:
        txt = u.text
        if "Peru" in txt:
            distrito = txt.split(",")[0].strip()  # solo el distrito
            break

    # Modalidad de trabajo
    modalidad = ""
    modos = driver.find_elements(By.TAG_NAME, "p")
    for m in modos:
        txt = m.text.strip()
        if txt in ["Presencial", "Remoto", "Híbrido"]:
            modalidad = txt
            break

    datos.append([i, link, job_title, distrito, modalidad])
    print(f"➡️ {job_title} | {distrito} | {modalidad}")

driver.quit()

# --- 4. Guardar en CSV final ---
with open("avisos_bumeran_detalle.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["N°", "Link", "Job Title", "Distrito", "Modalidad"])
    writer.writerows(datos)

print("\n💾 Se exportaron todos los avisos con Job Title, Distrito y Modalidad en 'avisos_bumeran_jobdetails.csv'")


🔎 Procesando aviso 1: https://www.bumeran.com.pe/empleos/software-engineer-senior-grupo-gloria-1117976663.html
➡️ Software Engineer Senior | La Victoria | Presencial

🔎 Procesando aviso 2: https://www.bumeran.com.pe/empleos/programador-frontend-senior-sonda-del-peru-s.a.-1117973000.html
➡️ Programador Frontend Senior | Lima | Remoto

🔎 Procesando aviso 3: https://www.bumeran.com.pe/empleos/analista-programador-java-1117972678.html
➡️ Analista Programador Java | Lima | Presencial

🔎 Procesando aviso 4: https://www.bumeran.com.pe/empleos/analista-programador-sede-ate-bumeran-selecta-1117969140.html
➡️ Analista Programador - Sede ATE | Ate | Presencial

🔎 Procesando aviso 5: https://www.bumeran.com.pe/empleos/trainee-programador-jr-building-software-1117968407.html
➡️ Trainee Programador Jr | Miraflores | Presencial

🔎 Procesando aviso 6: https://www.bumeran.com.pe/empleos/desarrollador-full-stack-node.js-c-java-angularjs-ibr-peru-s.a.-1117967655.html
➡️ Desarrollador Full Stack Node.js,

In [102]:

def scrape_jobs(driver, base_url, max_pages=3):
    jobs = []

    for page in range(1, max_pages + 1):
        print(f"📄 Página {page}")
        driver.get(base_url + f"?page={page}")
        time.sleep(8)

        job_cards = driver.find_elements(By.CSS_SELECTOR, "#listado-avisos > div > a > div")

        if not job_cards:
            print("⚠️ No hay más resultados.")
            break

        for job in job_cards:
            try:
                title = job.find_element(By.CSS_SELECTOR, "div.sc-dcOKER div:nth-child(1)").text.strip()
            except:
                title = "No especificado"

            try:
                district = job.find_element(By.CSS_SELECTOR, "div.sc-dcOKER div:nth-child(2)").text.strip()
            except:
                district = "No especificado"

            jobs.append({
                "Título": title,
                "Distrito": district
            })

    return pd.DataFrame(jobs)

# ---------- 5. Ejecutar scraping ----------
df = scrape_jobs(driver, base_url, max_pages=10)

# Guardar resultados en CSV
df.to_csv("bumeran_jobs.csv", index=False, encoding="utf-8-sig")
print("✅ Datos guardados en bumeran_jobs.csv")


📄 Página 1
📄 Página 2
📄 Página 3
📄 Página 4
📄 Página 5
📄 Página 6
⚠️ No hay más resultados.
✅ Datos guardados en bumeran_jobs.csv
